# 이상치 처리 기준 산정 : 각 항목들의 사분위값 범위의 최대값 도출

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import font_manager
import matplotlib.pyplot as plt
import os

## Grouping한 csv파일들 이름 load

In [2]:
csv_list1=[]
csv_list2=[]
csv_list3=[]
csv_list4=[]

def get_csv_list(db_name,csv_list):
    list1=os.listdir("../RawData/"+db_name+"/1hour")
    for i in range(len(list1)):
        list1[i]="../RawData/"+db_name+"/1hour/"+list1[i]
    csv_list+=list1
    return csv_list

In [3]:
csv_list1=get_csv_list('air_indoor_도서관',csv_list1)
csv_list2=get_csv_list('air_indoor_중학교',csv_list2)
csv_list3=get_csv_list('air_indoor_체육시설',csv_list3)
csv_list4=get_csv_list('air_indoor_초등학교',csv_list4)

In [4]:
print(csv_list1)
print(csv_list2)

['../RawData/air_indoor_도서관/1hour/ICW0W2000087.csv', '../RawData/air_indoor_도서관/1hour/ICW0W2000088.csv', '../RawData/air_indoor_도서관/1hour/ICW0W2000089.csv', '../RawData/air_indoor_도서관/1hour/ICW0W2000094.csv', '../RawData/air_indoor_도서관/1hour/ICW0W2000095.csv', '../RawData/air_indoor_도서관/1hour/ICW0W2000096.csv', '../RawData/air_indoor_도서관/1hour/ICW0W2000097.csv', '../RawData/air_indoor_도서관/1hour/ICW0W2000098.csv', '../RawData/air_indoor_도서관/1hour/ICW0W2000099.csv', '../RawData/air_indoor_도서관/1hour/ICW0W2000100.csv', '../RawData/air_indoor_도서관/1hour/ICW0W2000101.csv', '../RawData/air_indoor_도서관/1hour/ICW0W2000102.csv', '../RawData/air_indoor_도서관/1hour/ICW0W2000108.csv', '../RawData/air_indoor_도서관/1hour/ICW0W2000128.csv', '../RawData/air_indoor_도서관/1hour/ICW0W2000129.csv']
['../RawData/air_indoor_중학교/1hour/ICW0W2000010.csv', '../RawData/air_indoor_중학교/1hour/ICW0W2000011.csv', '../RawData/air_indoor_중학교/1hour/ICW0W2000013.csv', '../RawData/air_indoor_중학교/1hour/ICW0W2000014.csv', '../RawDat

## concatenate를 실행하여 시설별 데이터를 하나의 파일로 병합

In [5]:
def concat_all(db_name,csv_list):
    data=pd.DataFrame()
    
    for csv_file in csv_list:
        file=pd.read_csv(csv_file)
        file=pd.DataFrame(file)
        
        data=pd.concat([data,file])
        
    data.to_csv("../RawData/All_Grouped({:s}).csv".format(db_name),index=False)
    
    return data.columns

In [6]:
columms1=concat_all('air_indoor_도서관',csv_list1)
columms2=concat_all('air_indoor_중학교',csv_list2)
columms3=concat_all('air_indoor_체육시설',csv_list3)
columms4=concat_all('air_indoor_초등학교',csv_list4)

In [7]:
print(columms1)

for columm in columms1:
    print("{:s}_values = pd.to_numeric(file['{:s}'])".format(columm,columm))

Index(['time', 'in_co2', 'in_humi', 'in_pm10', 'in_temp', 'in_voc'], dtype='object')
time_values = pd.to_numeric(file['time'])
in_co2_values = pd.to_numeric(file['in_co2'])
in_humi_values = pd.to_numeric(file['in_humi'])
in_pm10_values = pd.to_numeric(file['in_pm10'])
in_temp_values = pd.to_numeric(file['in_temp'])
in_voc_values = pd.to_numeric(file['in_voc'])


## 병합된 각 파일들로 부터 사분위 범위 최대/최소값 계산

In [8]:
in_co2_lower=[]
in_pm10_lower=[]
in_voc_lower=[]

in_co2_upper=[]
in_pm10_upper=[]
in_voc_upper=[]

In [9]:
font_path = "./NanumGothicBold.ttf"
fontprop = font_manager.FontProperties(fname=font_path)

def get_boxplot_all(db_name,what,list1,list2):
    address = "../RawData/All_Grouped({:s}).csv".format(db_name)
    file = pd.read_csv(address)

    name = pd.to_numeric(file[what], errors='coerce')

    
    # Calculate Q1, Q3, and IQR
    Q1 = np.percentile(name.dropna(), 25)
    Q3 = np.percentile(name.dropna(), 75)
    IQR = Q3 - Q1

    # Calculate lower and upper bound
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    fig, ax = plt.subplots(figsize=(24, 24))
    sns.set_theme(style='whitegrid')
    sns.boxplot(data=name.dropna(), ax=ax)
    ax.set_title('The Distribution of ' + what, fontproperties=fontprop,fontsize=30,fontweight=1000)
    #ax.set_ylim(-10, 100)
    #plt.yticks((-10,0,10,20,30,40,50,60,70,80,90,100),fontsize=24,fontweight="bold")
    
    # Add text for lower and upper bound
    ax.text(0.02, lower_bound, f'Lower Bound: {lower_bound:.2f}', color='red',fontsize=36,fontweight="bold")
    ax.text(0.02, upper_bound, f'Upper Bound: {upper_bound:.2f}', color='red',fontsize=36,fontweight="bold")
    
    list1.append(lower_bound)
    list2.append(upper_bound)
    
    plt.savefig("../IQR/IQR({:s},{:s}).png".format(db_name,what))
    plt.close()
    
    return 

In [10]:
db_names=['air_indoor_도서관','air_indoor_중학교','air_indoor_체육시설','air_indoor_초등학교']
whats=['in_co2', 'in_pm10',  'in_voc']

list_lower=[in_co2_lower,in_pm10_lower,in_voc_lower]
list_upper=[in_co2_upper,in_pm10_upper,in_voc_upper]

for db_name in db_names:
    i=0
    for what in whats:
        get_boxplot_all(db_name,what,list_lower[i],list_upper[i])
        i+=1


#### 결과 : in_co2(811.33), in_pm10(60.84), in_voc(1665.09)

In [11]:
print("in_co2 : ",max(in_co2_upper))
print("in_pm10 : ",max(in_pm10_upper))
print("in_voc : ",max(in_voc_upper))

in_co2 :  811.3333333333334
in_pm10 :  60.83527542372882
in_voc :  1665.0862499999998
